In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from collections import Counter

# Delayed flights per airline, date
# Correlation between flight delay and origin airport, cancellation

In [3]:
airlines = pd.read_csv("/Users/maria.perensin/Documents/Data_Analytics_Bootcamp/Untitled Folder/archive (2)/airlines.csv")
airports = pd.read_csv("/Users/maria.perensin/Documents/Data_Analytics_Bootcamp/Untitled Folder/archive (2)/airports.csv")
flights = pd.read_csv("/Users/maria.perensin/Documents/Data_Analytics_Bootcamp/Untitled Folder/archive (2)/flights.csv")



/var/folders/8s/b_y1b6vs1gv0z8mhqsgz88480000gn/T/ipykernel_5268/3344653900.py:3: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  flights = pd.read_csv("/Users/maria.perensin/Documents/Data_Analytics_Bootcamp/Untitled Folder/archive (2)/flights.csv")


In [15]:
flights.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
flights[['WEATHER_DELAY']].isnull().value_counts()

WEATHER_DELAY
True             4755640
False            1063439
dtype: int64

In [21]:
flights[['YEAR']].nunique().value_counts()

1    1
dtype: int64

In [9]:
airlines.head()

,IATA_CODE,AIRLINE
0,UA,United Air Lines Inc.
1,AA,American Airlines Inc.
2,US,US Airways Inc.
3,F9,Frontier Airlines Inc.
4,B6,JetBlue Airways


In [22]:
data_2015= pd.read_csv("/Users/maria.perensin/Documents/Data_Analytics_Bootcamp/Untitled Folder/ten_years/2018.csv")

In [24]:
data_2015.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7213446 entries, 0 to 7213445
Data columns (total 28 columns):
 #   Column               Dtype  
---  ------               -----  
 0   FL_DATE              object 
 1   OP_CARRIER           object 
 2   OP_CARRIER_FL_NUM    int64  
 3   ORIGIN               object 
 4   DEST                 object 
 5   CRS_DEP_TIME         int64  
 6   DEP_TIME             float64
 7   DEP_DELAY            float64
 8   TAXI_OUT             float64
 9   WHEELS_OFF           float64
 10  WHEELS_ON            float64
 11  TAXI_IN              float64
 12  CRS_ARR_TIME         int64  
 13  ARR_TIME             float64
 14  ARR_DELAY            float64
 15  CANCELLED            float64
 16  CANCELLATION_CODE    object 
 17  DIVERTED             float64
 18  CRS_ELAPSED_TIME     float64
 19  ACTUAL_ELAPSED_TIME  float64
 20  AIR_TIME             float64
 21  DISTANCE             float64
 22  CARRIER_DELAY        float64
 23  WEATHER_DELAY        float64
 24

In [28]:
data_2015= data_2015.rename(columns={'CRS_DEP_TIME':'ESTIMATED_DEP_TIME', 'CRS_ARR_TIME':'ESTIMATED_ARR_TIME'})

In [38]:
data_2015 = data_2015.drop(columns=["TAXI_OUT", "WHEELS_OFF", "WHEELS_ON", "Unnamed: 27", "TAXI_IN", "AIR_TIME", "DIVERTED"])

In [36]:
#Next: split dates per month

In [41]:
data_2015 = data_2015.drop(columns=['OP_CARRIER_FL_NUM', 'CANCELLATION_CODE'])

In [43]:
data_2015['CANCELLED'].value_counts()

0.0    7096862
1.0     116584
Name: CANCELLED, dtype: int64

In [47]:
data_2015_date = data_2015['FL_DATE']

In [50]:
def extract_date(date):
    return date[5:7]

In [51]:
data_2015['MONTH'] = data_2015['FL_DATE'].map(extract_date)

In [52]:
data_2015.head()

,FL_DATE,OP_CARRIER,ORIGIN,DEST,ESTIMATED_DEP_TIME,DEP_TIME,DEP_DELAY,ESTIMATED_ARR_TIME,ARR_TIME,ARR_DELAY,CANCELLED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,MONTH
0,2018-01-01,UA,EWR,DEN,1517,1512.0,-5.0,1745,1722.0,-23.0,0.0,268.0,250.0,1605.0,NaN,NaN,NaN,NaN,NaN,01
1,2018-01-01,UA,LAS,SFO,1115,1107.0,-8.0,1254,1230.0,-24.0,0.0,99.0,83.0,414.0,NaN,NaN,NaN,NaN,NaN,01
2,2018-01-01,UA,SNA,DEN,1335,1330.0,-5.0,1649,1636.0,-13.0,0.0,134.0,126.0,846.0,NaN,NaN,NaN,NaN,NaN,01
3,2018-01-01,UA,RSW,ORD,1546,1552.0,6.0,1756,1754.0,-2.0,0.0,190.0,182.0,1120.0,NaN,NaN,NaN,NaN,NaN,01
4,2018-01-01,UA,ORD,ALB,630,650.0,20.0,922,936.0,14.0,0.0,112.0,106.0,723.0,NaN,NaN,NaN,NaN,NaN,01


In [57]:
new_columns = ['FL_DATE','MONTH', 'OP_CARRIER', 'ORIGIN', 'DEST', 'ESTIMATED_DEP_TIME',
       'DEP_TIME', 'DEP_DELAY', 'ESTIMATED_ARR_TIME', 'ARR_TIME', 'ARR_DELAY',
       'CANCELLED', 'CRS_ELAPSED_TIME', 'ACTUAL_ELAPSED_TIME', 'DISTANCE',
       'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY',
       'LATE_AIRCRAFT_DELAY']

data_2018= data_2015.reindex(columns=new_columns)

In [58]:
data_2018.head()

,FL_DATE,MONTH,OP_CARRIER,ORIGIN,DEST,ESTIMATED_DEP_TIME,DEP_TIME,DEP_DELAY,ESTIMATED_ARR_TIME,ARR_TIME,ARR_DELAY,CANCELLED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
0,2018-01-01,01,UA,EWR,DEN,1517,1512.0,-5.0,1745,1722.0,-23.0,0.0,268.0,250.0,1605.0,NaN,NaN,NaN,NaN,NaN
1,2018-01-01,01,UA,LAS,SFO,1115,1107.0,-8.0,1254,1230.0,-24.0,0.0,99.0,83.0,414.0,NaN,NaN,NaN,NaN,NaN
2,2018-01-01,01,UA,SNA,DEN,1335,1330.0,-5.0,1649,1636.0,-13.0,0.0,134.0,126.0,846.0,NaN,NaN,NaN,NaN,NaN
3,2018-01-01,01,UA,RSW,ORD,1546,1552.0,6.0,1756,1754.0,-2.0,0.0,190.0,182.0,1120.0,NaN,NaN,NaN,NaN,NaN
4,2018-01-01,01,UA,ORD,ALB,630,650.0,20.0,922,936.0,14.0,0.0,112.0,106.0,723.0,NaN,NaN,NaN,NaN,NaN


In [60]:
data_2018.to_csv("/Users/maria.perensin/Documents/Data_Analytics_Bootcamp/Untitled Folder/cleaned_data/data_2018.csv")

In [61]:
data_2015= pd.read_csv("/Users/maria.perensin/Documents/Data_Analytics_Bootcamp/Untitled Folder/ten_years/2015.csv")

In [62]:
data_2015= data_2015.rename(columns={'CRS_DEP_TIME':'ESTIMATED_DEP_TIME', 'CRS_ARR_TIME':'ESTIMATED_ARR_TIME'})
data_2015 = data_2015.drop(columns=['OP_CARRIER_FL_NUM', 'CANCELLATION_CODE',"TAXI_OUT", "WHEELS_OFF", "WHEELS_ON", "Unnamed: 27", "TAXI_IN", "AIR_TIME", "DIVERTED"])
data_2015['MONTH'] = data_2015['FL_DATE'].map(extract_date)

In [64]:
new_columns = ['FL_DATE','MONTH', 'OP_CARRIER', 'ORIGIN', 'DEST', 'ESTIMATED_DEP_TIME',
       'DEP_TIME', 'DEP_DELAY', 'ESTIMATED_ARR_TIME', 'ARR_TIME', 'ARR_DELAY',
       'CANCELLED', 'CRS_ELAPSED_TIME', 'ACTUAL_ELAPSED_TIME', 'DISTANCE',
       'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY',
       'LATE_AIRCRAFT_DELAY']

data_2015 = data_2015.reindex(columns=new_columns)

In [65]:
data_2015.to_csv("/Users/maria.perensin/Documents/Data_Analytics_Bootcamp/Untitled Folder/cleaned_data/data_2015.csv")

In [66]:
data_2016= pd.read_csv("/Users/maria.perensin/Documents/Data_Analytics_Bootcamp/Untitled Folder/ten_years/2016.csv")
data_2016= data_2016.rename(columns={'CRS_DEP_TIME':'ESTIMATED_DEP_TIME', 'CRS_ARR_TIME':'ESTIMATED_ARR_TIME'})
data_2016 = data_2016.drop(columns=['OP_CARRIER_FL_NUM', 'CANCELLATION_CODE',"TAXI_OUT", "WHEELS_OFF", "WHEELS_ON", "Unnamed: 27", "TAXI_IN", "AIR_TIME", "DIVERTED"])
data_2016['MONTH'] = data_2016['FL_DATE'].map(extract_date)

new_columns = ['FL_DATE','MONTH', 'OP_CARRIER', 'ORIGIN', 'DEST', 'ESTIMATED_DEP_TIME',
       'DEP_TIME', 'DEP_DELAY', 'ESTIMATED_ARR_TIME', 'ARR_TIME', 'ARR_DELAY',
       'CANCELLED', 'CRS_ELAPSED_TIME', 'ACTUAL_ELAPSED_TIME', 'DISTANCE',
       'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY',
       'LATE_AIRCRAFT_DELAY']

data_2016 = data_2016.reindex(columns=new_columns)

In [67]:
data_2016.to_csv("/Users/maria.perensin/Documents/Data_Analytics_Bootcamp/Untitled Folder/cleaned_data/data_2016.csv")

In [68]:
data_2017= pd.read_csv("/Users/maria.perensin/Documents/Data_Analytics_Bootcamp/Untitled Folder/ten_years/2017.csv")

In [69]:
data_2017= data_2017.rename(columns={'CRS_DEP_TIME':'ESTIMATED_DEP_TIME', 'CRS_ARR_TIME':'ESTIMATED_ARR_TIME'})
data_2017 = data_2017.drop(columns=['OP_CARRIER_FL_NUM', 'CANCELLATION_CODE',"TAXI_OUT", "WHEELS_OFF", "WHEELS_ON", "Unnamed: 27", "TAXI_IN", "AIR_TIME", "DIVERTED"])
data_2017['MONTH'] = data_2017['FL_DATE'].map(extract_date)

new_columns = ['FL_DATE','MONTH', 'OP_CARRIER', 'ORIGIN', 'DEST', 'ESTIMATED_DEP_TIME',
       'DEP_TIME', 'DEP_DELAY', 'ESTIMATED_ARR_TIME', 'ARR_TIME', 'ARR_DELAY',
       'CANCELLED', 'CRS_ELAPSED_TIME', 'ACTUAL_ELAPSED_TIME', 'DISTANCE',
       'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY',
       'LATE_AIRCRAFT_DELAY']

data_2017 = data_2017.reindex(columns=new_columns)

In [70]:
data_2017.to_csv("/Users/maria.perensin/Documents/Data_Analytics_Bootcamp/Untitled Folder/cleaned_data/data_2017.csv")